### DTS - Complete Streets

# 04a - Modal Composite Development - Step 1

**Author:** rmangan

---

**Purpose:**

Compile fields from DTS Sidewalk Inventory, Proposed Sidewalk Improvements, Proposed Sidewalk Additions, Current Bike Facilites, Proposed Bike Facilites, and Redevelopment Bike Facilities to DPP Street Centerline feature class. Datasets are compiled together either by directly joining on SegmentID, or by performing an Intersect, then joining the output using SegmentID

**This script performs the following functions:**

1. Develop "Modal Composite 01" - RCL + Ped data (Sidewalk Inventory, Sidewalk Improvements, Sidewalk Additions) centerline.
2. Develop "Modal Composite 02" - RCL + Ped + Bike data (Bike Existing, Bike Proposed, Bike Redevelopment)

**Global Assumptions and Notes:**
1. DPP Street Centerline downloaded 10/13/2020 from http://gis.hicentral.com/gis_layer_list_by_topic_category.html

**Non-Standard Python Modules utilized:**
1. arcpy 2.7 - used for geoprocessing

In [ ]:
# import modules
import arcpy
import os

In [ ]:
# set environment setttings
arcpy.env.workspace = "Z:\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_scratch.gdb"
arcpy.env.overwriteOutput = True

In [ ]:
# define variables
input_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb"

scratch_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_scratch.gdb"

output_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_output.gdb"


# Input Datasets
Ped_Plan = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\OPP_domain_processing.gdb\OPP_ModalPriority_081720_output"

Ped_Improve = os.path.join(input_gdb_path,"OPP_CandidateUpgrades_090420Dissolve_101320")

Ped_Add = os.path.join(input_gdb_path,"OPP_CandidateWalkways_091520Dissolve_update101620")

Bike_Exist = os.path.join(input_gdb_path,"BIKEPLAN_Existing_Bikeways")

Bike_Proposed = os.path.join(input_gdb_path,"BIKEPLAN_Proposed_Bikeways")

Bike_ReDev = os.path.join(input_gdb_path,"BIKEPLAN_Redevelopment_Bikeways")

RCL = os.path.join(input_gdb_path,"RCL_Public_Street_Centerline_20201013")

In [ ]:
## Functions

def isUniqueValueField(dataset, field):
    #Check if a field is unique for a given dataset, return True/False
    idList = []
    with arcpy.da.SearchCursor(dataset, field) as cursor:
        for row in cursor:
            idList.append(row[0])
    if len(idList) != len(set(idList)):
        return False
    else:
        return True


def LineLength(dataset):
    #Add line length field & calculate length. Used to validate geometry is the same after joining
    #THIS FUNCTION MODIFIES IT'S INPUT DATASET    
    arcpy.AddField_management(dataset, "line_length", "DOUBLE")
    arcpy.CalculateGeometryAttributes_management(dataset,[["line_length", "LENGTH"]])
    
    
    
def CheckMultiPart(dataset):
    #check if a feature class contains multipart geometry, report OIDs of multipart geometry to table
    #THIS FUNCTION MODIFIES IT'S INPUT DATASET 
    
    #get count of input features
    input_count = arcpy.GetCount_management(dataset)
    print(str(input_count) + " records in input dataset")

    #add tmpUID field to input dataset
    arcpy.AddField_management(dataset, "tmpUID","LONG")
    print("tmpUID field added to input dataset")

    #determine OID field of input dataset
    OID_field_name = arcpy.Describe(dataset).OIDFieldName
    print("OID = " + str(OID_field_name))

    #calculate OID to tmpUID
    print("Calculating OID to tmpUID...")
    arcpy.CalculateField_management(dataset,"tmpUID","!" + OID_field_name +"!")
    print("OID calculated to tmpUID")

    #define singlepart dataset
    singlepart_fc_name = str(dataset)+"_singlepart"
    singlepart_fc = os.path.join(scratch_gdb_path, singlepart_fc_name)

    #delete singlepart dataset if it already exists
    if arcpy.Exists(singlepart_fc):
        print("Singlepart dataset already exists, deleting...")
        arcpy.Delete_management(singlepart_fc)
        print("Singlepart dataset deleted")

    #split input features into singlepart dataset
    print("Splitting input dataset into singlepart feature...")
    arcpy.MultipartToSinglepart_management(dataset,singlepart_fc)
    print("input exploded to singlepart" + singlepart_fc_name)

    #get feature count of output 
    output_count = arcpy.GetCount_management(singlepart_fc)
    print(str(output_count)+ " records in singlepart FC")

    # if multipart features found, run freq and get ID of multipart feature
    if input_count != output_count:
        print("Multipart features found.")
        print("{0} records in input, {1} records in singlepart output.".format(input_count, output_count))

        #define frequency table
        freq_table_name = str(dataset)+"_freq"
        freq_table = os.path.join(scratch_gdb_path, freq_table_name)

        #delete freq table if it already exists
        if arcpy.Exists(freq_table):
            print("freq table exists, deleting...")
            arcpy.Delete_management(freq_table)
            print('freq table deleted')

        #run frequency analysis on singlepart dataset
        print("Running frequency analysis...")
        arcpy.Frequency_analysis(singlepart_fc, freq_table, "tmpUID")
        print("frequency analysis complete")

        #print out report of multipart features found
        with arcpy.da.SearchCursor(freq_table, ['FREQUENCY','tmpUID'],'"FREQUENCY" > 1') as cursor:
            for row in cursor:
                print("Multipart feature found at OID: {}".format(row[1]))

    else:
        print("No multipart features found.")

## 01 - Develop Modal Composite 01
### Load Ped Plan attributes to Street Centerline
---

Create Modal Composite 01 by loading Ped Plan attributes (Sidewalk Inventory, Ped Improvements, Ped Additions) to DPP Road Centerline.


**This section performs the following functions:**

1. Add temporary line length fields to input dataset
2. Check input datasets for multipart geometry
3. Join Ped Sidewalk Inventory to RCL on SegmentID
4. Validate joined segments by comparing temporary line length fields

**Assumptions and Notes:**
1. Field aliases of input Sidewalk Inventory dataset must already be assigned.
2. Ped Improvements and Ped Additions datasets are not geometrically identical to RCL. Join is performed by:
    1. Spatially intersecting the datasets together to create join key
    2. Joining the intersect output back to RCL on SegmentID

In [ ]:
#step 1

#copy feature classes to scratch gdb for processing
print("Copying RCL to scratch gdb...")
RCL_temp = arcpy.CopyFeatures_management(RCL,(os.path.join(scratch_gdb_path, "RCL_temp")))
print("Done")

print("Copying Ped Data to scratch gdb...")
ped_temp = arcpy.CopyFeatures_management(Ped_Plan,(os.path.join(scratch_gdb_path, "Ped_temp2")))
print("Done")

#add & calculate line_length fields
print("Adding line length fields to datasets...")
LineLength(RCL_temp)
LineLength(ped_temp)
print("Done")

#check for multipart geometry
CheckMultiPart(RCL_temp)
CheckMultiPart(ped_temp)

print("done")

In [ ]:
#step 2

#copy RCL_temp to serve as join target and as output modal_composite_01 dataset
print("Copying RCL_temp to serve as modal composite_01 join target...")
modal_composite_01 = arcpy.CopyFeatures_management(RCL_temp,(os.path.join(scratch_gdb_path, "modal_composite_01")))
print("Copying complete.")

#join fields from ped_temp to modal_composite 01
join_target = modal_composite_01
join_target_field = "SEGMENTID"
join_table = ped_temp
join_table_field = "SEGMENTID"
join_fields = [
    "NBRightPed","SBLeftPedZ","NBRightP_1","SBLeftPe_1","NBRightP_2",
    "SBLeftPe_2","NBRightP_3","SBLeftPe_3","NBRightP_4","SBLeftPe_4",
    "NBRightP_5","SBLeftPedB","NBRightP_6","SBLeftPe_5","NBRightP_7",
    "SBLeftPe_6","NBRightP_8","SBLeftPe_7","NBRightVis","SBLeftVisu",
    "MidBlkXwkU","MidBlkXwkC","MidBlkXw_1","TraffCalmi","PPN_Final",
    "HPI","PpnNotCity","line_length"
]

print("Joining fields...")
arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")

#assign domains to fields
#store field/domain mapping in a dict, field name is key, domain table is value
ped_domain_mapping = {
    "SBLeftPe_6": "OPP_domain_PedBuffRdBuff",
    "NBRightP_8": "OPP_domain_PedBuffLight",
    "NBRightP_7": "OPP_domain_PedBuffRdBuff",
    "SBLeftPe_5": "OPP_domain_PedBuffTree",
    "NBRightP_6": "OPP_domain_PedBuffTree",
    "SBLeftPedB": "OPP_domain_PedBuffFrn",
    "NBRightP_5": "OPP_domain_PedBuffFrn",
    "SBLeftPe_4": "OPP_domain_PedZoneInt",
    "NBRightP_4": "OPP_domain_PedZoneInt",
    "SBLeftPe_3": "OPP_domain_PedZoneCond",
    "NBRightP_3": "OPP_domain_PedZoneCond",
    "SBLeftPe_2": "OPP_domain_PedZoneWidth",
    "NBRightP_2": "OPP_domain_PedZoneDrvycut",
    "SBLeftPe_1": "OPP_domain_PedZoneWidth",
    "NBRightP_1": "OPP_domain_PedZoneWidth",
    "SBLeftPedZ": "OPP_domain_PedZoneType",
    "NBRightPed": "OPP_domain_PedZoneType",
    "SBLeftPe_7": "OPP_domain_PedBuffLight",
    "NBRightVis": "OPP_domain_VisualInterest",
    "SBLeftVisu": "OPP_domain_VisualInterest",
    "MidBlkXwkU": "OPP_domain_MidBlkXwkUse",
    "MidBlkXwkC": "OPP_domain_MidBlkXwkCond",
    "MidBlkXw_1": "OPP_domain_MidBlkXwkCntrl",
    "TraffCalmi": "OPP_domain_TraffCalming"}

#iterate through domain dict and assign domains to newly joined fields
for key, value in ped_domain_mapping.items():
    print("assigning domain {} to field {}".format(value,key))
    print("Done")
    arcpy.management.AssignDomainToField(modal_composite_01, key, value)    
print("Domain Assignment Complete")


#print out report of different line length segments (only needed if multipart QC was run)
with arcpy.da.SearchCursor(modal_composite_01, ['OBJECTID','line_length', 'line_length_1'],'"line_length" <> "line_length_1"') as cursor:
    for row in cursor:
        print("Length difference found at OID: {}".format(row[0]))
        
print("done")

In [ ]:
#step 3 -join validation (optional)
#modal features with no join targes are stored in ped_temp_join_validation as "SEGMENTID_1" IS NULL

#copy ped_temp for join validation
print("Copying ped_temp to validate join...")
ped_temp_join_validation = arcpy.CopyFeatures_management(ped_temp,(os.path.join(scratch_gdb_path, "ped_temp_join_validation")))

# join rcl to modal
join_target = ped_temp_join_validation
join_target_field = "SEGMENTID"
join_table = RCL_temp
join_table_field = "SEGMENTID"
join_fields = ["SEGMENTID"]

print("Joining fields...")
arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")

#print out report of modal features with no valid join targets
with arcpy.da.SearchCursor(ped_temp_join_validation, ['OBJECTID','SEGMENTID', 'SEGMENTID_1'],'"SEGMENTID_1" IS NULL') as cursor:
    for row in cursor:
        print("No join targets in RCL for ObjectID: {}, SEGMENTID: {}".format(row[0],row[1]))
        
print("done")

In [ ]:
#step 4 - join Ped Plan - Upgrades & New Features to RCL

# inputs are not geometrically identical to RCL (new sidewalk data spans multiple RCL segments)
# join performed via spatial intersect to split lines by segements & assigning segmentIDs for join key

#copy inputs to scratch gdb for processing
print("Copying Ped Improve to scratch gdb...")
Ped_Improve_temp = arcpy.CopyFeatures_management(Ped_Improve,(os.path.join(scratch_gdb_path, "Ped_Improve_temp")))
print("Done")

print("Copying Ped Upgrade to scratch gdb...")
Ped_Add_temp = arcpy.CopyFeatures_management(Ped_Add,(os.path.join(scratch_gdb_path, "Ped_Add_temp")))
print("Done")

#intesect Ped datsets with RCL
print("Running Ped Improvement intersect...")
Ped_Improve_intersect = arcpy.Intersect_analysis([Ped_Improve_temp, RCL_temp],(os.path.join(scratch_gdb_path, "Ped_Improve_intersect")))
print("Done")

print("Running Ped Add intersect...")
Ped_Add_intersect = arcpy.Intersect_analysis([Ped_Add_temp, RCL_temp],(os.path.join(scratch_gdb_path, "Ped_Add_RCL_intersect")))
print("Done")

#join target
join_target = modal_composite_01
join_target_field = "SEGMENTID"

#join Ped Improvement Interserct Results to RCL...
print("Joining Ped Improvement fields...")
join_table = Ped_Improve_intersect
join_table_field = "SEGMENTID"
join_fields = ["FID_Ped_Improve_temp",
               "ProjectID",
               "Extents"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")

#join Ped Additions Intersect Results to RCL...
print("Joining Ped Additions fields...")
join_table = Ped_Add_intersect
join_table_field = "SEGMENTID"
join_fields = ["FID_Ped_Add_temp",
               "ProjectID",
               "Extents"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")

#modify fields that imported with duplicate names
renamed_fields = {"ProjectID":"Ped_Improve_ProjectID",
                  "Extents":"Ped_Immprove_Extents",
                  "ProjectID_1":"Ped_Add_Project_ID",
                  "Extents_1":"Ped_Add_Extents"}

for key, value in renamed_fields.items():
    print("renaming field {} to field {}".format(key,value))
    arcpy.AlterField_management(modal_composite_01, key, value)

print("done")

print("Modal Composite 01 - Complete")

## 02 - Develop Modal Composite 02
### Load Bike Plan attributes onto Street Centerline

---

Create Modal Composite 02 by loading Bike Plan (existing, proposed, redevelopment) attributes to Modal Composite 01


**This section performs the following functions:**

1. Copy DTS inputs to scratch gdb for processing
2. Assign field aliases to bike fields using dictionary
3. Intersect bike datasets with RCL
4. Join intersect outputs back to modal_composite_01 to create modal_composite_02

**Assumptions and Notes:**
1. 

In [ ]:
#step 1 - copy inputs to scratch gdb and pre-process each dataset prior to joining to RCL

#copy Bike Plan Datsets to scratch for processing
print("Copying Bike Existing to scratch gdb...")
Bike_Exist_temp = arcpy.CopyFeatures_management(Bike_Exist,(os.path.join(scratch_gdb_path, "Bike_Exist_temp")))
print("Done")

print("Copying Bike Proposed to scratch gdb...")
Bike_Proposed_temp = arcpy.CopyFeatures_management(Bike_Proposed,(os.path.join(scratch_gdb_path, "Bike_Proposed_temp")))
print("Done")

print("Copying Bike ReDev to scratch gdb...")
Bike_ReDev_temp = arcpy.CopyFeatures_management(Bike_ReDev,(os.path.join(scratch_gdb_path, "Bike_ReDev_temp")))
print("Done")

#store field, field aliases mappings in a dict
bike_exist_alias = {
    "Fac_Name":"Facility Name (Bike Existing)",
    "Fac_Desc":"Facility Description (Bike Existing)",
    "Fac_Type":"Facility Type (Bike Existing)",
    "Length_mi":"Length (miles) (Bike Existing)",   
    "Owner":"Owner (Bike Existing)",
    "DP_area":"DP Area (Bike Existing)"
}

bike_proposed_alias = {
    "Project_ID":"Project ID (Bike Proposed)",
    "Fac_Name":"Facility Name (Bike Proposed)",
    "Fac_Desc":"Facility Description (Bike Proposed)",
    "Fac_Type":"Facility Type (Bike Proposed)",
    "length_mi":"Length (miles) (Bike Proposed)",
    "Owner":"Owner (Bike Proposed)",
    "DP_area":"DP Area (Bike Proposed)",
    "Priority":"Priority (Bike Proposed)",
    "Cost_Est":"Cost Estimate (Bike Proposed)"
}

bike_redev_alias = {
    "ProjectID":"Project ID (Bike ReDev)",
    "Fac_Name":"Facility Name (Bike ReDev)",
    "Fac_Desc":"Facitlity Description (Bike ReDev)",
    "Fac_Type":"Facility Type (Bike ReDev)",
    "Length_mi":"Length (miles) (Bike ReDev)",
    "Owner":"Owner (Bike ReDev)",
    "DP_area":"DP Area (Bike ReDev)"    
}

#loop throug dicts and assign aliases & rename fields for each dataset

#bike existing
for key, value in bike_exist_alias.items():
    new_field_name = str(key)+"_BE"
    print("assigning alias {} to field {}, field renamed to {}".format(value,key,new_field_name))
    arcpy.AlterField_management(Bike_Exist_temp,key,new_field_name, value)    
print("Done")

#bike proposed
for key, value in bike_proposed_alias.items():
    new_field_name = str(key)+"_BP"
    print("assigning alias {} to field {}, field renamed to {}".format(value,key,new_field_name))
    arcpy.AlterField_management(Bike_Proposed_temp,key,new_field_name, value)    
print("Done")

#bike redevelopment
for key, value in bike_redev_alias.items():
    new_field_name = str(key)+"_BR"
    print("assigning alias {} to field {}, field renamed to {}".format(value,key,new_field_name))
    arcpy.AlterField_management(Bike_ReDev_temp,key,new_field_name, value)    
print("Done")
                   
print("Bike input processing Done")

In [ ]:
#step 02 - intersect datasets with RCL and load attributes from output by joining on SegmentID

#intesect bike datsets with RCl
print("Running Bike Exist intersect...")
Bike_Exist_RCL_intersect = arcpy.Intersect_analysis([Bike_Exist_temp, RCL_temp],(os.path.join(scratch_gdb_path, "Bike_Exist_RCL_intersect")))

print("Running Bike Proposed intersect...")
Bike_Proposed_RCL_intersect = arcpy.Intersect_analysis([Bike_Proposed_temp, RCL_temp],(os.path.join(scratch_gdb_path, "Bike_Proposed_RCL_intersect")))

print("Running Bike ReDev intersect...")
Bike_ReDev_RCL_intersect = arcpy.Intersect_analysis([Bike_ReDev_temp, RCL_temp],(os.path.join(scratch_gdb_path, "Bike_ReDev_RCL_intersect")))
print("Done")
   
#join fields from resultant to modal composite

#create modal_composite 02 from  modal_composite_01
print("Copying modal_composite_01 to serve as modal composite join target...")
modal_composite_02 = arcpy.CopyFeatures_management(modal_composite_01,(os.path.join(scratch_gdb_path, "modal_composite_02")))
print("Copying complete.")

#reassign domains (may not be needed)
#store field/domain mapping in a dict, field name is key, domain table is value
ped_domain_mapping = {
    "SBLeftPe_6": "OPP_domain_PedBuffRdBuff",
    "NBRightP_8": "OPP_domain_PedBuffLight",
    "NBRightP_7": "OPP_domain_PedBuffRdBuff",
    "SBLeftPe_5": "OPP_domain_PedBuffTree",
    "NBRightP_6": "OPP_domain_PedBuffTree",
    "SBLeftPedB": "OPP_domain_PedBuffFrn",
    "NBRightP_5": "OPP_domain_PedBuffFrn",
    "SBLeftPe_4": "OPP_domain_PedZoneInt",
    "NBRightP_4": "OPP_domain_PedZoneInt",
    "SBLeftPe_3": "OPP_domain_PedZoneCond",
    "NBRightP_3": "OPP_domain_PedZoneCond",
    "SBLeftPe_2": "OPP_domain_PedZoneWidth",
    "NBRightP_2": "OPP_domain_PedZoneDrvycut",
    "SBLeftPe_1": "OPP_domain_PedZoneWidth",
    "NBRightP_1": "OPP_domain_PedZoneWidth",
    "SBLeftPedZ": "OPP_domain_PedZoneType",
    "NBRightPed": "OPP_domain_PedZoneType",
    "SBLeftPe_7": "OPP_domain_PedBuffLight",
    "NBRightVis": "OPP_domain_VisualInterest",
    "SBLeftVisu": "OPP_domain_VisualInterest",
    "MidBlkXwkU": "OPP_domain_MidBlkXwkUse",
    "MidBlkXwkC": "OPP_domain_MidBlkXwkCond",
    "MidBlkXw_1": "OPP_domain_MidBlkXwkCntrl",
    "TraffCalmi": "OPP_domain_TraffCalming"}

#iterate through domain dict and assign domains to newly joined fields
for key, value in ped_domain_mapping.items():
    print("assigning domain {} to field {}".format(value,key))
    print("Done")
    arcpy.management.AssignDomainToField(modal_composite_02, key, value)
    
print("Domain Assignment Complete")
                                        
#join fields from Bike Existing to modal_composite test layer
join_target = modal_composite_02
join_target_field = "SEGMENTID"

#join Bike Existing intersect results
print("Joining Bike Existing fields...")
join_table = Bike_Exist_RCL_intersect
join_table_field = "SEGMENTID"
join_fields = ["Fac_Name_BE",
               "Fac_Desc_BE",
               "DP_area_BE",
               "Fac_Type_BE",
               "Length_mi_BE",
               "Owner_BE"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")

#join Bike Proposed intersect results
print("Joining Bike Proposed fields...")
join_table = Bike_Proposed_RCL_intersect
join_table_field = "SEGMENTID"
join_fields = ["Fac_Name_BP",
               "Fac_Desc_BP",
               "length_mi_BP",
               "DP_area_BP",
               "Project_ID_BP",
               "Fac_Type_BP",
               "Priority_BP"
               "Owner_BP",
               "Cost_Est_BP"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")

#join Bike ReDev intersect results
print("Joining Bike ReDev fields...")
join_table = Bike_ReDev_RCL_intersect
join_table_field = "SEGMENTID"
join_fields = ["Fac_Name_BR",
               "Fac_Desc_BR",
               "Project_ID_BR",
               "DP_area_BR",
               "Length_mi_BR",
               "Fac_Type_BR",
               "Owner_BR"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")

print("Bike Processing Complete")


## QC functions

In [ ]:
#list records of different line length

#print out report of multipart features found
with arcpy.da.SearchCursor(ped_temp_join_validation, ['OBJECTID','SEGMENTID', 'SEGMENTID_1'],'"SEGMENTID_1" IS NULL') as cursor:
    for row in cursor:
        print("No join targets in RCL for ObjectID: {}, SEGMENTID: {}".format(row[0],row[1]))

In [ ]:
# PED re-copy to scratch - rerun as needed

#1) copy feature classes to scratch gdb for processing
ped_temp = arcpy.CopyFeatures_management(Ped_Plan,(os.path.join(scratch_gdb_path, "Ped_temp")))

#2) add & calculate line_length fields
LineLength(ped_temp)

## 1986 ROW dataset field parsing (MOVE TO DIFFERENT NOTEBOOK)

**Summary**  
blah blah blah

**Details**  
blah blah blah


**Assumptions**  
blah blah blah

**Code Snippet**  
> code goes here

**Links**  
[enter link description here](www.google.com)



In [ ]:
#create num fields from 1986 text fields & parse value to new fields handle text entry via hardcoded mapping

#copy input dataset while testing functions
print("copying modal input for processing...")
modal_processing_temp = arcpy.CopyFeatures_management(modal_composite_03,"modal_processing_temp")
print("done")

#store field, field alias in dict
new_fields = {"ln_exist_num":"1986 Existing Lanes (num)",
              "ln_prop_num": "1986 Proposed Lanes (num)",
              "row_exist_num":"1986 Existing ROW (num)",
              "row_prop_num":"1986 Proposed ROW (num)"}

#loop through dict and add fields
for key,value in new_fields.items():
    print("Adding field {0}".format(key))
    arcpy.AddField_management(modal_processing_temp,field_name=key,field_type="SHORT", field_alias = value)
          
print("Field additions complete")


In [ ]:
#field to expose to update cursor
fields = ["ln_exist","ln_prop","row_exist","row_prop","ln_exist_num","ln_prop_num", "row_exist_num","row_prop_num"]

#update ln_exist_num w/ update cursor
print("parse ln_exist to int")
with arcpy.da.UpdateCursor(modal_processing_temp,fields,'NOT "ln_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if row[0] == '4+ bus':
                row[4] = 4
            elif row[0] == '3+ bus':
                row[4] = 3
            else:
                row[4]= int(row[0])
            cursor.updateRow(row)
        except ValueError as error:
            print(error)            
print("ln_exist parsed\n")
            
#update ln_prop_num w/ update cursor
print("parse ln_prop to int")
with arcpy.da.UpdateCursor(modal_processing_temp,fields,'NOT "ln_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if row[1] == '5+ bus':
                row[5] = 5
            else:
                row[5]= int(row[1])
            cursor.updateRow(row)
        except ValueError as error:
            print(error)
print("ln_prop parsed\n")
            
#update row_exist_num w/ update cursor
print("parse row_exist to num")
with arcpy.da.UpdateCursor(modal_processing_temp,fields,'NOT "ln_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if row[2] == 'Var. to 80':
                row[6] = 80
            elif row[2] == 'Var. 60-70':
                row[6] = 65
            elif row[2] == 'Var. 36-50':
                row[6] == 61
            elif row[2] == 'Var. 14-26':
                row[6] = 20
            elif row[2] == '86/100':
                row[6] = 93
            elif row[2] == '60-90':
                row[6] = 75
            elif row[2] == '60-76':
                row[6] = 68
            elif row[2] == '60-70':
                row[6] = 65
            elif row[2] == '60-64':
                row[6] = 62
            elif row[2] == '50-80':
                row[6] = 65
            elif row[2] == '50-60':
                row[6] = 55
            elif row[2] == '44-56':
                row[6] = 50
            elif row[2] == '40/50':
                row[6] = 45
            elif row[2] == '40-56':
                row[6] = 48
            elif row[2] == '30-50':
                row[6] = 40
            elif row[2] == '30-40':
                row[6] = 35
            elif row[2] == '25-50':
                row[6] = 38
            elif row[2] == '20-40':
                row[6] = 30
            elif row[2] == '25-50':
                row[6] = 38
            elif row[2] == '120-140':
                row[6] = 130
            elif row[2] == '113.5':
                row[6] = 114
            elif row[2] == '100-110':
                row[6] = 105
            elif row[2] == '0':
                row[6] = None
            else:
                row[6]= int(row[2])
            cursor.updateRow(row)
        except ValueError as error:
            print(error)
print("row_exist parsed\n")            
                  
#update row_prop_num w/ update cursor
print("parse row_prop to int")
with arcpy.da.UpdateCursor(modal_processing_temp,fields,'NOT "ln_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:          
            if row[3] == 'Var. to 80':
                row[7] = 80
            elif row[3] == 'Var. 14-26':
                row[7] = 20
            elif row[3] == '86/100':
                row[7] = 93
            elif row[3] == '60-90':
                row[7] = 75
            elif row[3] == '60-76':
                row[7] = 68
            elif row[3] == '60-70':
                row[7] = 65
            elif row[3] == '60-64':
                row[7] = 62
            elif row[3] == '56-80':
                row[7] = 68
            elif row[3] == '50-60':
                row[7] = 55
            elif row[3] == '40/50':
                row[7] = 45
            elif row[3] == '30-40':
                row[7] = 35
            elif row[3] == '25-50':
                row[7] = 38
            elif row[3] == '120-140':
                row[7] = 130
            elif row[3] == '113.5':
                row[7] = 114
            elif row[3] == '100-110':
                row[7] = 105
            else:
                row[7] = int(row[3])
            cursor.updateRow(row)
        except ValueError as error:
            print(error)
print("row_prop parsed\n")

print("Processing Complete")